In [ ]:
import pretty_midi
import numpy as np
import matplotlib.pyplot as plt
import os
import math
import numpy as np
np.set_printoptions(threshold=np.inf)

import pydub
from pydub import AudioSegment

import librosa
import random
# pretty midi: https://craffel.github.io/pretty-midi/#pretty-midi-timesignature

In [ ]:
avilable_list=[]
# this list record the availavle data we can used for both midi and wav

In [ ]:
##filter the data not available for midi
work_dir = r'C:\Users\Arnold\Desktop\4560\CNN\dataset\mid-aud-to-midi-bearaudiotool.com-mp3-to-midi'

available_list=[]

for parent, dirnames, filenames in os.walk(work_dir,  followlinks=True):
    for filename in filenames:
        if filename.endswith(".mid"):
            file_path = os.path.join(parent, filename)
            midi_data = pretty_midi.PrettyMIDI(file_path)
            proll = midi_data.instruments[0].get_piano_roll(fs=100).transpose()
            #fs: Sampling frequency of the columns, i.e. each column is spaced apart by 1./fs seconds
            index = os.path.splitext(os.path.splitext(filename)[0])[0]
            
            print(index,filename,len(midi_data.instruments),proll.shape)
            
            if (proll.shape[0]>4550)or (proll.shape[0]<4450):
                print("error too long or too short")
                continue
            
            print("available")
            available_list.append(index)

In [ ]:
#filter the data not available for wav
work_dir = r'C:\Users\Arnold\Desktop\4560\CNN\dataset\audio'
output_dir = r'C:\Users\Arnold\Desktop\4560\CNN\dataset\wav-audio'
AudioSegment.converter = r'E:\ffmpeg\bin\ffmpeg.exe'

print(len(available_list))
for parent, dirnames, filenames in os.walk(work_dir,  followlinks=True):
    for filename in filenames:
        if filename.endswith(".mp3"):
            index = os.path.splitext(filename)[0]
            print(index)
            if index in available_list:
                wavsong = AudioSegment.from_mp3(os.path.join(parent, filename))
                if len(wavsong)<45000:
                    available_list.remove(index)
                    
print(len(available_list))

In [ ]:
# random sampling 400+100
random.seed(42)
sample_data = random.sample(available_list,500) # randomly sample 500 data
train_list = sample_data[0:400] # 400 as train
test_list = sample_data[400:] # 100 as test
print(train_list,"\n",test_list)
with open(r'C:\Users\Arnold\Desktop\4560\CNN\dataset\400-100-dataset\list.json', 'w+') as f:
    json.dump({'train': train_list,'test':test_list},f)

In [ ]:
# open the lable csv
import json
arousal_dict={}
with open(r'C:\Users\Arnold\Desktop\4560\CNN\dataset\v-a values\arousal.json','r') as load_f:
    arousal_dict = json.load(load_f)
    
with open(r'C:\Users\Arnold\Desktop\4560\CNN\dataset\v-a values\volence.json','r') as load_f:
    volence_dict = json.load(load_f)

#validata our loading
print(arousal_dict["2"]['5'],volence_dict["10"]['37'])

In [ ]:
def extraFeature_wav(parent_path,filename,output_dir,x_data,y_a,y_v,arousal_dict,volence_dict,frame_size,step):
    wavsong = AudioSegment.from_mp3(os.path.join(parent_path, filename))

    one_data = []

    for i in range(60):
        outputpath = os.path.join(output_dir,index+'-'+str(i)+".wav")
        wavsong[(15000+500*i):(15500+500*i)].export(outputpath, format="wav")
        y, sr = librosa.load(outputpath, sr=None)
        os.remove(outputpath)

        fragment = np.array(
                        np.log(
                            librosa.feature.melspectrogram(
                                y, sr, n_mels=128,n_fft=int(sr*frame_size), hop_length=int(sr*step)
                            )
                        ).transpose())
        fragment = fragment[np.newaxis,:]

        if one_data==[]:
            one_data = fragment.copy()                    
        else:
            one_data = np.vstack([one_data,fragment.copy()])

    #packaged one data
    one_data = one_data[np.newaxis,:]
    if x_data == []:
        x_data = one_data.copy()
    else:
        x_data = np.vstack([x_data,one_data.copy()])
        
    for i in range(60):
        y_a.append(arousal_dict[index][str(i)])
        y_v.append(volence_dict[index][str(i)])
        
        
    return x_data,y_a,y_v

In [ ]:
# then we will build 4 dataset: wav+cnn, wav+crnn, midi+cnn, wav+crnn

# wav+cnn, shape = (400+100)*3000*128
# We should rewrite as a function...um
AudioSegment.converter = r'E:\ffmpeg\bin\ffmpeg.exe'
work_dir = r'C:\Users\Arnold\Desktop\4560\CNN\dataset\audio'
output_dir = r'C:\Users\Arnold\Desktop\4560\CNN\dataset\wav-audio'

x_train_data = []
x_test_data =[]
y_a_train_data = []
y_a_test_data = []
y_v_train_data = []
y_v_test_data = []

for parent, dirnames, filenames in os.walk(work_dir,  followlinks=True):
    for filename in filenames:
        if filename.endswith(".mp3"):
            index = os.path.splitext(filename)[0]
            if index in train_list:
                print("train",index)
                (x_train_data,y_a_train_data,y_v_train_data) = extraFeature_wav(parent,filename,output_dir,
                                                x_train_data,y_a_train_data,y_v_train_data,
                                                arousal_dict,volence_dict,
                                                0.025,0.010)
                print(x_train_data.shape,"train",index)
                
                
            
            if index in test_list:
                (x_test_data,y_a_test_data,y_v_test_data) = extraFeature_wav(parent,filename,output_dir,
                                                x_test_data,y_a_test_data,y_v_test_data,
                                                arousal_dict,volence_dict,
                                                0.025,0.010)
                    
                print(x_test_data.shape,"test",index)

In [ ]:
#save the wav+cnn

class NumpyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return json.JSONEncoder.default(self, obj)
    
with open(r'C:\Users\Arnold\Desktop\4560\CNN\dataset\400-100-dataset\wav_cnn.json', 'w+') as f:
    json.dump({'x_train':x_train_data,
               'x_test':x_test_data,
               'y_a_train':y_a_train_data,
               'y_a_test':y_a_test_data,
               'y_v_train':y_v_train_data,
               'y_v_test':y_v_test_data},f,cls=NumpyEncoder)

In [ ]:
# wav+crnn
# wav+crnn, shape = (400+100)*60*128
AudioSegment.converter = r'E:\ffmpeg\bin\ffmpeg.exe'
work_dir = r'C:\Users\Arnold\Desktop\4560\CNN\dataset\audio'
output_dir = r'C:\Users\Arnold\Desktop\4560\CNN\dataset\wav-audio'

x_train_data = []
x_test_data =[]
y_a_train_data = []
y_a_test_data = []
y_v_train_data = []
y_v_test_data = []

for parent, dirnames, filenames in os.walk(work_dir,  followlinks=True):
    for filename in filenames:
        if filename.endswith(".mp3"):
            index = os.path.splitext(filename)[0]
            if index in train_list:
                (x_train_data,y_a_train_data,y_v_train_data) = extraFeature_wav(parent,filename,output_dir,
                    x_train_data,y_a_train_data,y_v_train_data,
                    arousal_dict,volence_dict,
                    0.5,1)
                print(x_train_data.shape,"train",index)
                
                
            
            if index in test_list:
                (x_test_data,y_a_test_data,y_v_test_data) = extraFeature_wav(parent,filename,output_dir,
                    x_test_data,y_a_test_data,y_v_test_data,
                    arousal_dict,volence_dict,
                    0.5,1)
                    
                print(x_test_data.shape,"test",index)

In [ ]:
#save the wav+crnn


class NumpyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return json.JSONEncoder.default(self, obj)
    
with open(r'C:\Users\Arnold\Desktop\4560\CNN\dataset\400-100-dataset\wav_crnn.json', 'w+') as f:
    json.dump({'x_train':x_train_data,
               'x_test':x_test_data,
               'y_a_train':y_a_train_data,
               'y_a_test':y_a_test_data,
               'y_v_train':y_v_train_data,
               'y_v_test':y_v_test_data},f,cls=NumpyEncoder)

In [ ]:
def extraFeature_midi_cnn(parent,filename,output_dir,x_data,y_a,y_v,arousal_dict,volence_dict,fs,num_frame):
    file_path = os.path.join(parent, filename)
    midi_data = pretty_midi.PrettyMIDI(file_path)
    proll = midi_data.instruments[0].get_piano_roll(fs=fs).transpose()
    #fs: Sampling frequency of the columns, i.e. each column is spaced apart by 1./fs seconds
    index = os.path.splitext(os.path.splitext(filename)[0])[0]
            
    print(index,filename,len(midi_data.instruments),proll.shape)
            
    if proll.shape[0] > 90*num_frame:
        proll = proll[:90*num_frame,:128]
    while proll.shape[0]< 90*num_frame:
        lastrow = proll[proll.shape[0]-1].copy()
        proll=np.vstack([proll,lastrow])
    proll = proll.reshape(90,num_frame,128)
    proll = proll[30:90]
    proll = proll[np.newaxis,:]
            
    if x_data==[]:
        x_data = proll.copy()
    else:
        x_data = np.vstack([x_data,proll.copy()])
                
    arousal_data = []
    for i in range(0,60):
        arousal_data.append(arousal_dict[index][str(i)])
    y_a.append(arousal_data)
            
    volence_data = []
    for i in range(0,60):
        volence_data.append(volence_dict[index][str(i)])
    y_v.append(volence_data)
        
        
    return x_data,y_a,y_v

In [ ]:
# midi+cnn
work_dir = r'C:\Users\Arnold\Desktop\4560\CNN\dataset\mid-aud-to-midi-bearaudiotool.com-mp3-to-midi'

x_train_data = []
x_test_data =[]
y_a_train_data = []
y_a_test_data = []
y_v_train_data = []
y_v_test_data = []

for parent, dirnames, filenames in os.walk(work_dir,  followlinks=True):
    for filename in filenames:
        if filename.endswith(".mid"):
            index = os.path.splitext(os.path.splitext(filename)[0])[0]
            if index in train_list:
                (x_train_data,y_a_train_data,y_v_train_data)=extraFeature_midi_cnn(
                        parent,filename,output_dir,
                        x_train_data,y_a_train_data,y_v_train_data,
                        arousal_dict,volence_dict,
                        fs=100,num_frame=50)
                print(x_train_data.shape,"train",index)
            if index in test_list:
                (x_test_data,y_a_test_data,y_v_test_data)=extraFeature_midi_cnn(
                        parent,filename,output_dir,
                        x_test_data,y_a_test_data,y_v_test_data,
                        arousal_dict,volence_dict,
                        fs=100,num_frame=50)
                print(x_test_data.shape,"test",index)

In [ ]:
#save the midi+cnn


class NumpyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return json.JSONEncoder.default(self, obj)
    
with open(r'C:\Users\Arnold\Desktop\4560\CNN\dataset\400-100-dataset\midi_cnn.json', 'w+') as f:
    json.dump({'x_train':x_train_data,
               'x_test':x_test_data,
               'y_a_train':y_a_train_data,
               'y_a_test':y_a_test_data,
               'y_v_train':y_v_train_data,
               'y_v_test':y_v_test_data},f,cls=NumpyEncoder)

In [ ]:
#midi+crnn
work_dir = r'C:\Users\Arnold\Desktop\4560\CNN\dataset\mid-aud-to-midi-bearaudiotool.com-mp3-to-midi'

x_train_data = []
x_test_data =[]
y_a_train_data = []
y_a_test_data = []
y_v_train_data = []
y_v_test_data = []

for parent, dirnames, filenames in os.walk(work_dir,  followlinks=True):
    for filename in filenames:
        if filename.endswith(".mid"):
            index = os.path.splitext(os.path.splitext(filename)[0])[0]
            if index in train_list:
                (x_train_data,y_a_train_data,y_v_train_data)=extraFeature_midi_cnn(
                        parent,filename,output_dir,
                        x_train_data,y_a_train_data,y_v_train_data,
                        arousal_dict,volence_dict,
                        fs=2,num_frame=1)
                print(x_train_data.shape,"train",index)
            if index in test_list:
                (x_test_data,y_a_test_data,y_v_test_data)=extraFeature_midi_cnn(
                        parent,filename,output_dir,
                        x_test_data,y_a_test_data,y_v_test_data,
                        arousal_dict,volence_dict,
                        fs=2,num_frame=1)
                print(x_test_data.shape,"test",index)

In [ ]:
#save the midi+cnn
class NumpyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return json.JSONEncoder.default(self, obj)
    
with open(r'C:\Users\Arnold\Desktop\4560\CNN\dataset\400-100-dataset\midi_crnn.json', 'w+') as f:
    json.dump({'x_train':x_train_data,
               'x_test':x_test_data,
               'y_a_train':y_a_train_data,
               'y_a_test':y_a_test_data,
               'y_v_train':y_v_train_data,
               'y_v_test':y_v_test_data},f,cls=NumpyEncoder)

In [ ]:
#validate
